In [5]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [7]:
#define sentences
sentences = ["Programming is a good habit",
            "This is bad code you must improve it",
            "This is a perfect idea",
            "Your code was clean and neat",
            "You are excellent programmer",
            "You have performed badly in the examinations",
            "You could have made it better",
            "Good work! Keep it up.",
            "Poor performance",
            "Improve yourself"]
#define remarks
remarks = array([1,0,1,1,1,0,0,1,0,0])

## Tokenization and one_hot_encoding

Our approach involves breaking down each sentence into individual words and representing each sequence as a vector using one-hot encoding.

In [8]:
# integer encode the sentences
vocabulary_size = 50
encoded_sequence = [ one_hot(s, vocabulary_size) for s in sentences ]
print( encoded_sequence )

[[35, 39, 4, 44, 21], [47, 39, 8, 20, 37, 42, 40, 33], [47, 39, 4, 48, 22], [23, 20, 14, 46, 43, 48], [37, 25, 39, 44], [37, 27, 14, 36, 15, 23, 16], [37, 36, 27, 10, 33, 42], [44, 11, 49, 33, 39], [47, 9], [40, 28]]



The function called "one_hot" will require a sentence and a vocabulary_size as input parameters. The code shown above will use the "one_hot" function imported from Keras to convert all the sentences into an encoded form consisting of integers.

After performing the one_hot function on the sentences, the above result is the output.

But we can see that all the one_hot_encoded vectors are not of equal length, so to overcome this problem, we pad every sentence using the pad_sequences function. 

## Padding

The belowe code will pad every sequence. Following is the output after applying the pad_sqeuences function on the one_hot_encoded vectors. We can see that every sequence has a length of ten numbers. 

Number of zeros added to each vector = max_length - length of one_hot_encoded vecto

In [9]:
# pad documents to a max length of 10 words
max_length = 10
padded_sequence = pad_sequences( encoded_sequence, maxlen = max_length, padding='pre' )
print( padded_sequence )

[[ 0  0  0  0  0 35 39  4 44 21]
 [ 0  0 47 39  8 20 37 42 40 33]
 [ 0  0  0  0  0 47 39  4 48 22]
 [ 0  0  0  0 23 20 14 46 43 48]
 [ 0  0  0  0  0  0 37 25 39 44]
 [ 0  0  0 37 27 14 36 15 23 16]
 [ 0  0  0  0 37 36 27 10 33 42]
 [ 0  0  0  0  0 44 11 49 33 39]
 [ 0  0  0  0  0  0  0  0 47  9]
 [ 0  0  0  0  0  0  0  0 40 28]]


Till now we have created proper input data that we can feed to the embedding layer.

## Embedding layers


In the code below we are creating embedding layers.

In [10]:
# define the model
model = Sequential()
embedding_layer = Embedding( vocabulary_size, 10, input_length = max_length )
model.add( embedding_layer )
model.add( Flatten() )
model.add( Dense(1, activation='sigmoid') )
# compile the model
model.compile( optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'] )
# summarize the model
print( model.summary() )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 10)            500       
                                                                 
 flatten (Flatten)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________
None


We are creating a sequential model. In the embedding layer, vocabulary_size is input_dim parameter, output_dim is set to 10, and input_length of the sequence is the length of the padded sequence (max_length), i.e., 10. After flattening it apply a dense layer using the sigmoid activation function. 

Compile the model using the adam optimizer and use the binary_crossentropy as a loss function. 

The summary of the model is displayed above.

## Train and evaluate the model

In [11]:
# fit the model 

#The padded_sequence and remarks are then passed through this embedding layer. 
#Further accuracy of the model is evaluated
model.fit( padded_sequence, remarks, epochs=50, verbose=0 )
# evaluate the model
loss, accuracy = model.evaluate( padded_sequence, remarks, verbose=0 )
print( 'Accuracy: %f' % (accuracy*100) )

2023-05-11 13:51:08.597778: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Accuracy: 100.000000
